In [1]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install openai==0.28

     ---------------------------------------- 0.0/66.9 kB ? eta -:--:--
     ---------------------------------------- 66.9/66.9 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 76.5/76.5 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/381.6 kB ? eta -:--:--
   --------------------------------------  378.9/381.6 kB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 381.6/381.6 kB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/63.0 kB ? eta -:--:--
   ---------------------------------------- 63.0/63.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
   ---------------------------------------- 51.6/51.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/90.2 kB ? eta -:--:--
   ---------------------------------------- 90.2/90.2 kB ? eta 0:00:00
   ------------------------------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import openai
openai.api_key = ''

In [16]:
llm = 'gpt-3.5-turbo'
def get_completion(prompt, model=llm):

    messages = [{
        #role : (시스템 프롬프트), (chat), (나의질문)
        #content : 내가 질문하고자 하는 내용
        'role' : 'user',
        'content' : prompt
    }]
    #model = 어떤 모델에게 질문할래?
    # message = 어떤 내용을, 누가 질문하니?
    # temperature = 0(정석답변), -1(가장 창의적)
    responses = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature=0
    )

    return responses.choices[0].message['content']

In [ ]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [18]:
!pip install langchain==0.1.0

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB ? eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could t

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
#langchain을 중매자로 단톡을 팜
from langchain.chat_models import ChatOpenAI
#위의 "템플릿"을 적용합니다.
from langchain.prompts import ChatPromptTemplate

In [25]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [26]:
#openai한테 말 걸기 전에 어떤 내용을 전달할지를 세팅
#{}를 통해 template의 특정 부분을 "변수화" 할 수 있다
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [ ]:

#카톡방 개설 : llm한테, 농담 "0" 의 정도로 얘기하는 카톡방을 만들자

message = ChatOpenAI(temperature=0, model = llm, openai_api_key = '')

#카톡 메세지 보냄.(위에서 정의한 prompt 양식을 통해서 text 라는 변수에 customer_review를 전달해서 보내줌)
content = prompt_template.format_messages(text = customer_review)

#openai가 보낸 답장
response = message(content)

print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


In [31]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [32]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [33]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
#프롬프트 템플릿 => chatgpt 에게 할 질문의 형태를 정해놓고
#아웃풋 파서 -> 응답을 이렇게 받겠다
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""
#다시한번약식
prompt = ChatPromptTemplate.from_template(template=review_template_2)
#다시만들어진 양식에 변수화된 format_inseturctions을 제공
messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [35]:
response = message(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}
```


In [36]:
output_dict = output_parser.parse(response.content)
print(type(output_dict))


<class 'dict'>


In [37]:
output_dict.get('gift')

True